# Intro to MAUP
@author: eveomett
AI for Redistricting, USF
All data retrieved 11/21/23: <br>
    [https://redistrictingdatahub.org/dataset/illinois-block-pl-94171-2020-by-table/](https://redistrictingdatahub.org/dataset/illinois-block-pl-94171-2020-by-table/) <br>
    [https://redistrictingdatahub.org/dataset/vest-2020-illinois-precinct-and-election-results/](https://redistrictingdatahub.org/dataset/vest-2020-illinois-precinct-and-election-results/) <br>
    [https://redistrictingdatahub.org/dataset/2021-illinois-congressional-districts-approved-plan/](https://redistrictingdatahub.org/dataset/2021-illinois-congressional-districts-approved-plan/)

In [173]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair

In [174]:
maup.progress.enabled = True

## Import and Explore the Data

This file is useful for the data below:<br>
[https://www2.census.gov/programs-surveys/decennial/2020/technical-documentation/complete-tech-docs/summary-file/2020Census_PL94_171Redistricting_StatesTechDoc_English.pdf](https://www2.census.gov/programs-surveys/decennial/2020/technical-documentation/complete-tech-docs/summary-file/2020Census_PL94_171Redistricting_StatesTechDoc_English.pdf)

### Population

Note: importing the census data takes 4-5 minutes per file.  The other fi#### This first census file has population, Hispanic and non-Hispanic details.les are faster. <br>

#### This first census file has total population of different races.

In [175]:
start_time = time.time()
population1_df = gpd.read_file("./il_pl2020_b/il_pl2020_p1_b.shp")
end_time = time.time()
print("The time to import il_pl2020_p1_b.shp is:",
      (end_time-start_time)/60, "mins")

The time to import il_pl2020_p1_b.shp is: 1.8158757170041402 mins


#### This second census file has population, Hispanic and non-Hispanic details.

In [176]:
start_time = time.time()
population2_df = gpd.read_file("./il_pl2020_b/il_pl2020_p2_b.shp")
end_time = time.time()
print("The time to import il_pl2020_p2_b.shp is:",
      (end_time-start_time)/60, "mins")

The time to import il_pl2020_p2_b.shp is: 1.5123519817988078 mins


In [177]:
population2_df = population2_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])

In [178]:
population_df = pd.merge(population1_df, population2_df, on='GEOID20')

In [179]:
population1_df.columns

Index(['GEOID20', 'SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'P0010001',
       'P0010002', 'P0010003', 'P0010004', 'P0010005', 'P0010006', 'P0010007',
       'P0010008', 'P0010009', 'P0010010', 'P0010011', 'P0010012', 'P0010013',
       'P0010014', 'P0010015', 'P0010016', 'P0010017', 'P0010018', 'P0010019',
       'P0010020', 'P0010021', 'P0010022', 'P0010023', 'P0010024', 'P0010025',
       'P0010026', 'P0010027', 'P0010028', 'P0010029', 'P0010030', 'P0010031',
       'P0010032', 'P0010033', 'P0010034', 'P0010035', 'P0010036', 'P0010037',
       'P0010038', 'P0010039', 'P0010040', 'P0010041', 'P0010042', 'P0010043',
       'P0010044', 'P0010045', 'P0010046', 'P0010047', 'P0010048', 'P0010049',
       'P0010050', 'P0010051', 'P0010052', 'P0010053', 'P0010054', 'P0010055',
       'P0010056', 'P0010057', 'P0010058', 'P0010059', 'P0010060', 'P0010061',
       'P0010062', 'P0010063', 'P0010064', 'P0010065', 'P0010066', 'P0010067',
       'P0010068', 'P0010069', 'P0010070', 'P0010071', 'geom

hispanic = total - non-hispanic

In [180]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['P0020005'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['P0020006'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['P0020007'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['P0020008'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['P0020009'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['P0020010'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['P0020011'], 1)

This third census file has voting age population (VAP), Hispanic and non-Hispanic details.

In [181]:
start_time = time.time()
vap_df= gpd.read_file("./il_pl2020_b/il_pl2020_p4_b.shp")
end_time = time.time()
print("The time to import il_pl2020_p4_b.shp is:",
      (end_time-start_time)/60, "mins")

The time to import il_pl2020_p4_b.shp is: 1.270311999320984 mins


## 2020 election data

The data set below has 2020 presidential election results by precinct

In [182]:
start_time = time.time()
vest20 = gpd.read_file("./il_vest_20/il_vest_20.shp")
end_time = time.time()
print("The time to import il_vest_20.shp is:",
      (end_time-start_time)/60, "mins")

The time to import il_vest_20.shp is: 0.03718810081481934 mins


## Put data in same geometry units

Here, we'll assign blocks to precints <br>

In [183]:
blocks_to_precincts_assignment = maup.assign(population_df.geometry, vest20.geometry)
vap_blocks_to_precincts_assignment = maup.assign(vap_df.geometry, vest20.geometry)

100%|█████████████████████████████████████████| 10083/10083 [00:02<00:00, 3418.82it/s]
/opt/anaconda3/lib/python3.9/site-packages/maup/intersections.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > area_cutoff].reset_index(drop=True)
/opt/anaconda3/lib/python3.9/site-packages/maup/intersections.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/opt/anaconda3/lib/python3.9/site-packages/maup/assign.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, ar

The columns below are the ones we're interested in.

In [184]:
pop_column_names = ['P0020001', 'P0020002', 'P0020005', 'P0020006', 'P0020007', 'P0020008', 'P0020009', 'P0020010', 'P0020011', 
                    'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE']

In [185]:
vap_column_names = ['P0040001', 'P0040002', 'P0040005', 'P0040006', 'P0040007', 'P0040008', 'P0040009', 'P0040010', 'P0040011']

We'll put all of the population columns into the election dataframe

In [186]:
vest20[pop_column_names] = population_df[pop_column_names].groupby(blocks_to_precincts_assignment).sum()
vest20[vap_column_names] = vap_df[vap_column_names].groupby(vap_blocks_to_precincts_assignment).sum()

Let's check to make sure we didn't lose anyone. 

The method maup.doctor() outputs true if geometries look OK.  False if there are gaps or overlaps

In [187]:
print(maup.doctor(vest20))

100%|██████████████████████████████████████████| 10083/10083 [00:11<00:00, 846.85it/s]
/opt/anaconda3/lib/python3.9/site-packages/maup/repair.py:331: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlaps = inters[inters.area > 0].make_valid()


True


## Congressional District data

The data set below is a shapefile of the congressional districts

In [188]:
start_time = time.time()
cong_df = gpd.read_file("./il_cong_adopted_2021/HB 1291 FA #1.shp")
end_time = time.time()
print("The time to import HB 1291 FA #1.shp is:",
      (end_time-start_time)/60, "mins")

The time to import HB 1291 FA #1.shp is: 0.003995637098948161 mins


In [189]:
precincts_to_districts_assignment = maup.assign(vest20.geometry, cong_df.geometry)
vest20["CD"] = precincts_to_districts_assignment

100%|█████████████████████████████████████████████████| 17/17 [00:24<00:00,  1.45s/it]
/opt/anaconda3/lib/python3.9/site-packages/maup/intersections.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > area_cutoff].reset_index(drop=True)
/opt/anaconda3/lib/python3.9/site-packages/maup/intersections.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/opt/anaconda3/lib/python3.9/site-packages/maup/assign.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, ar

In [190]:
print(set(vest20["CD"]))
# for precinct_index in range(len(vest20)):
#     vest20.at[precinct_index, "CD"] = cong_df.at[vest20.at[precinct_index, "CD"], district_col_name]
vest20['CD'] = vest20['CD'].apply(lambda t: t + 1)
print(set(cong_df[district_col_name]))
print(set(vest20["CD"]))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}


## Senate House data

In [191]:
send = gpd.read_file("./il_sldu_2021/il_sldu_2021.shp")

In [192]:
precincts_to_send_assignment = maup.assign(vest20.geometry, send.geometry)
vest20["SEND"] = precincts_to_send_assignment.apply(lambda t: t + 1)

100%|█████████████████████████████████████████████████| 59/59 [00:12<00:00,  4.88it/s]
/opt/anaconda3/lib/python3.9/site-packages/maup/intersections.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > area_cutoff].reset_index(drop=True)
/opt/anaconda3/lib/python3.9/site-packages/maup/intersections.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/opt/anaconda3/lib/python3.9/site-packages/maup/assign.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, ar

## State house data

In [193]:
hdist = gpd.read_file("./il_sldl_adopted_2021/il_sldl_adopted_2021.shp")

Use maup.quick_repair() or maup.smart_repair() if there are problems.\
Before smart_repair(), we need to use gdf.estimate_utm_crs() to change it to a utm crs.\
Then, use maup.doctor() to make sure that the overlap is fixed.

In [194]:
hdist = hdist.to_crs(hdist.estimate_utm_crs())

In [195]:
hdist = smart_repair(hdist)

/opt/anaconda3/lib/python3.9/site-packages/maup/smart_repair.py:95: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  geometries_df["geometry"][i] = shapely.wkb.loads(
/opt/anaconda3/lib/python3.9/site-packages/maup/smart_repair.py:149: FutureWa

Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.


/opt/anaconda3/lib/python3.9/site-packages/maup/smart_repair.py:372: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  pieces_df["polygon indices"][i] = set()


Identifying overlaps...


  0%|                                                         | 0/119 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.9/site-packages/maup/smart_repair.py:415: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  pieces_df["polygon indices"][i] = pieces

Resolving overlaps...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]


In [196]:
maup.doctor(hdist)

100%|██████████████████████████████████████████████| 118/118 [00:00<00:00, 317.68it/s]


True

In [197]:
# it was 4326 so we have to change it to 4269 first
hdist = hdist.to_crs('EPSG:4269')

In [198]:
precincts_to_hdist_assignment = maup.assign(vest20.geometry, hdist.geometry)

100%|███████████████████████████████████████████████| 118/118 [00:06<00:00, 19.02it/s]
/opt/anaconda3/lib/python3.9/site-packages/maup/intersections.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > area_cutoff].reset_index(drop=True)
/opt/anaconda3/lib/python3.9/site-packages/maup/intersections.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/opt/anaconda3/lib/python3.9/site-packages/maup/assign.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, ar

In [199]:
vest20["HDIST"] = precincts_to_hdist_assignment.apply(lambda t: t + 1)

Get districts assignment and put it into dataframe

We will rename columns by convention.  For example, see: <br>
[https://github.com/mggg-states/PA-shapefiles](https://github.com/mggg-states/PA-shapefiles)

In [200]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
                    'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER', 'P0020011': 'NH_2MORE',
                    'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
                                        'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP', 'P0040011': '2MOREVAP'}

In [201]:
vest20.rename(columns=rename_dict, inplace = True)

In [202]:
vest20.columns

Index(['STATEFP20', 'COUNTYFP20', 'VTDST20', 'GEOID20', 'NAME20', 'G20PREDBID',
       'G20PRERTRU', 'G20PRELJOR', 'G20PREGHAW', 'G20PREACAR', 'G20PRESLAR',
       'G20USSDDUR', 'G20USSRCUR', 'G20USSIWIL', 'G20USSLMAL', 'G20USSGBLA',
       'geometry', 'TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN',
       'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'H_WHITE', 'H_BLACK',
       'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP',
       'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP',
       '2MOREVAP', 'CD', 'SEND', 'HDIST'],
      dtype='object')

The other candidates are from other parties.  We'll drop them . . . 

### Rename the election columns

In [203]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [204]:
original_col = vest20.columns[5:16]
new_col = [rename(i, '20') for i in original_col]
rename_dict = dict(zip(original_col, new_col))
vest20 = vest20.rename(columns=rename_dict)

add all the other party column to one column with suffix "O" for other

In [205]:
vest20 = vest20.groupby(level=0, axis=1).sum()

/var/folders/87/cd_m04yx2nqfk8k9ljzt2xc80000gn/T/ipykernel_33670/738118509.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  vest20 = vest20.groupby(level=0, axis=1).sum()


In [206]:
vest20.columns

Index(['2MOREVAP', 'AMINVAP', 'ASIANVAP', 'BVAP', 'CD', 'COUNTYFP20',
       'GEOID20', 'HDIST', 'HISP', 'HVAP', 'H_2MORE', 'H_AMIN', 'H_ASIAN',
       'H_BLACK', 'H_NHPI', 'H_OTHER', 'H_WHITE', 'NAME20', 'NHPIVAP',
       'NH_2MORE', 'NH_AMIN', 'NH_ASIAN', 'NH_BLACK', 'NH_NHPI', 'NH_OTHER',
       'NH_WHITE', 'OTHERVAP', 'PRE20D', 'PRE20O', 'PRE20R', 'SEND',
       'STATEFP20', 'TOTPOP', 'USS20D', 'USS20O', 'USS20R', 'VAP', 'VTDST20',
       'WVAP', 'geometry'],
      dtype='object')

Do some checks to make sure that the population values are nearly the same in each district

In [207]:
print(vest20.loc[vest20["CD"] == 1, "TOTPOP"].sum())
print(vest20.loc[vest20["CD"] == 2, "TOTPOP"].sum())
pop_vals = [vest20.loc[vest20["CD"] == n, "TOTPOP"].sum() for n in range(1, 18)]
print(pop_vals)

758905
749190
[758905, 749190, 754269, 754380, 754242, 756238, 745883, 752200, 754527, 753930, 753542, 756658, 753908, 752848, 752707, 757898, 751183]


In [208]:
election_df = gpd.GeoDataFrame(vest20, crs="EPSG:4269")

### 2018 IL Election Data Projected to 2020 VTDs

In [209]:
vest18 = gpd.read_file('./il_vest_18/il_vest_18.shp')

In [210]:
vest18.columns

Index(['STATEFP20', 'COUNTYFP20', 'VTDST20', 'GEOID20', 'NAME20', 'G18GOVDPRI',
       'G18GOVRRAU', 'G18GOVCMCC', 'G18GOVLJAC', 'G18ATGDRAO', 'G18ATGRHAR',
       'G18ATGLHAR', 'G18SOSDWHI', 'G18SOSRHEL', 'G18SOSLDUT', 'G18COMDMEN',
       'G18COMRSEN', 'G18COMLBAL', 'G18TREDFRE', 'G18TRERDOD', 'G18TRELLEH',
       'geometry'],
      dtype='object')

In [211]:
vest18 = gpd.read_file('./il_vest_18/il_vest_18.shp')
original_col = vest18.columns[5:-1]
new_col = [rename(i, '18') for i in original_col]
rename_dict = dict(zip(original_col, new_col))
vest18 = vest18.rename(columns=rename_dict)
vest18 = vest18.groupby(level=0, axis=1).sum()
col_name = list(set(new_col))
col_name.sort()

/var/folders/87/cd_m04yx2nqfk8k9ljzt2xc80000gn/T/ipykernel_33670/2417531403.py:6: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  vest18 = vest18.groupby(level=0, axis=1).sum()


In [212]:
vest18 = gpd.GeoDataFrame(vest18, crs="EPSG:4269")

In [213]:
# convert 2018 pricinct to block
vest18_to_block_assginment = maup.assign(vest18.geometry, population_df.geometry)

100%|███████████████████████████████████████| 369978/369978 [03:49<00:00, 1611.03it/s]
/opt/anaconda3/lib/python3.9/site-packages/maup/intersections.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > area_cutoff].reset_index(drop=True)
/opt/anaconda3/lib/python3.9/site-packages/maup/intersections.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/opt/anaconda3/lib/python3.9/site-packages/maup/assign.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, ar

In [214]:
block18 = population_df[['geometry']]

In [215]:
block18[col_name] = vest18[col_name].groupby(vest18_to_block_assginment).sum()

In [216]:
clock18_to_pricinct_assginment = maup.assign(block18.geometry, election_df.geometry)

100%|█████████████████████████████████████████| 10083/10083 [00:03<00:00, 3321.66it/s]
/opt/anaconda3/lib/python3.9/site-packages/maup/intersections.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > area_cutoff].reset_index(drop=True)
/opt/anaconda3/lib/python3.9/site-packages/maup/intersections.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/opt/anaconda3/lib/python3.9/site-packages/maup/assign.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, ar

In [217]:
election_df[col_name] = vest18[col_name].groupby(clock18_to_pricinct_assginment).sum()

In [218]:
election_df = election_df.groupby(level=0, axis=1).sum()

/var/folders/87/cd_m04yx2nqfk8k9ljzt2xc80000gn/T/ipykernel_33670/834950066.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  election_df = election_df.groupby(level=0, axis=1).sum()


In [222]:
election_df = election_df[[
    'STATEFP20',
    'COUNTYFP20',
    'VTDST20',
    'GEOID20',
    'NAME20',
    'CD',
    'SEND',
    'HDIST',
    'TOTPOP',
    'NH_2MORE',
    'NH_AMIN',
    'NH_ASIAN',
    'NH_BLACK',
    'NH_NHPI',
    'NH_OTHER',
    'NH_WHITE',
    'HISP',
    'H_AMIN',
    'H_ASIAN',
    'H_BLACK',
    'H_NHPI',
    'H_OTHER',
    'H_WHITE',
    'H_2MORE',
    'VAP',
    'HVAP',
    'WVAP',
    'BVAP',
    'AMINVAP',
    'ASIANVAP',
    'NHPIVAP',
    'OTHERVAP',
    '2MOREVAP',
    'ATG18D',
    'ATG18R',
    'ATG18O',
    'PRE20D',
    'PRE20R',
    'PRE20O',
    'SOS18D',
    'SOS18R',
    'SOS18O',
    'COM18D',
    'COM18R',
    'COM18O',
    'GOV18D',
    'GOV18R',
    'GOV18O',
    'TRE18D',
    'TRE18R',
    'TRE18O',
    'USS20D',
    'USS20R',
    'USS20O',
    'geometry'
]]

In [224]:
election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")

In [225]:
election_df.to_file("./IL/IL.shp")

shp_file = gpd.read_file('./IL/IL.shp')

shp_file.to_file('./IL/IL.geojson', driver='GeoJSON')

In [226]:
from gerrychain import Graph

#Only do once to build json and read from file when generating ensembles
graph = Graph.from_file("./IL/IL.shp", ignore_errors=True)
graph.to_json("./IL/IL.json")

/opt/anaconda3/lib/python3.9/site-packages/gerrychain/graph/graph.py:262: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
